In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

In [35]:
df=pd.read_csv("/content/Swiggy.csv")

In [36]:
df

,date,favorite_count,followers_count,friends_count,full_text,retweet_count,retweeted,screen_name,tweet_id,user_id
0,7/18/2019 22:47,0,82,219,@Mahi_2510 @swiggy_in The ultimate answer will...,0,False,syamantak1,1,"6,19,59,419"
1,7/18/2019 22:43,0,102,129,@SwiggyCares I hope so.,0,False,Bharatbbhushn,2,"14,32,84,383"
2,7/18/2019 22:37,0,102,129,@SwiggyCares I think you have the order detail...,0,False,Bharatbbhushn,3,"14,32,84,383"
3,7/18/2019 22:35,0,13,16,"@swiggy_in People are complaining here also, i...",0,False,taifkhalid01,4,"8,30,34,00,00,00,00,00,000"
4,7/18/2019 22:25,0,102,129,@SwiggyCares Do you even know the meaning of y...,0,False,Bharatbbhushn,5,"14,32,84,383"
...,...,...,...,...,...,...,...,...,...,...
16707,07-11-2019 12:13,0,0,2,@SwiggyCares they said as delivery box is not ...,0,False,UmmeMohammad18,"16,708","11,44,99,00,00,00,00,00,000"
16708,07-11-2019 12:12,0,106,146,"@swiggy_in Deliver to karta nahi, install kyo ...",0,False,Desi_Life1,"16,709","10,39,41,00,00,00,00,00,000"
16709,07-11-2019 12:11,0,91,248,@swiggy_in thanks for spilling my drink. I wai...,0,False,Pinks_Forever_,"16,710","1,70,09,14,573"
16710,07-11-2019 12:11,0,0,2,@SwiggyCares hello there i use swigggy regular...,0,False,UmmeMohammad18,"16,711","11,44,99,00,00,00,00,00,000"


In [37]:
df.head()

,date,favorite_count,followers_count,friends_count,full_text,retweet_count,retweeted,screen_name,tweet_id,user_id
0,7/18/2019 22:47,0,82,219,@Mahi_2510 @swiggy_in The ultimate answer will...,0,False,syamantak1,1,"6,19,59,419"
1,7/18/2019 22:43,0,102,129,@SwiggyCares I hope so.,0,False,Bharatbbhushn,2,"14,32,84,383"
2,7/18/2019 22:37,0,102,129,@SwiggyCares I think you have the order detail...,0,False,Bharatbbhushn,3,"14,32,84,383"
3,7/18/2019 22:35,0,13,16,"@swiggy_in People are complaining here also, i...",0,False,taifkhalid01,4,"8,30,34,00,00,00,00,00,000"
4,7/18/2019 22:25,0,102,129,@SwiggyCares Do you even know the meaning of y...,0,False,Bharatbbhushn,5,"14,32,84,383"


In [38]:
df.tail()

,date,favorite_count,followers_count,friends_count,full_text,retweet_count,retweeted,screen_name,tweet_id,user_id
16707,07-11-2019 12:13,0,0,2,@SwiggyCares they said as delivery box is not ...,0,False,UmmeMohammad18,"16,708","11,44,99,00,00,00,00,00,000"
16708,07-11-2019 12:12,0,106,146,"@swiggy_in Deliver to karta nahi, install kyo ...",0,False,Desi_Life1,"16,709","10,39,41,00,00,00,00,00,000"
16709,07-11-2019 12:11,0,91,248,@swiggy_in thanks for spilling my drink. I wai...,0,False,Pinks_Forever_,"16,710","1,70,09,14,573"
16710,07-11-2019 12:11,0,0,2,@SwiggyCares hello there i use swigggy regular...,0,False,UmmeMohammad18,"16,711","11,44,99,00,00,00,00,00,000"
16711,07-11-2019 12:09,0,54,163,@Being_Humor @ZomatoIN @swiggy_in can you help...,0,False,TejasShelar2498,"16,712","7,17,51,10,00,00,00,00,000"


In [39]:
df.shape

(16712, 10)

In [40]:
df.columns

Index(['date', 'favorite_count', 'followers_count', 'friends_count',
       'full_text', 'retweet_count', 'retweeted', 'screen_name', 'tweet_id',
       'user_id'],
      dtype='object')

In [41]:
#descriptive statistics
df.describe()

,favorite_count,followers_count,friends_count,retweet_count
count,16712.000000,1.671200e+04,16712.000000,16712.000000
mean,0.554691,3.156743e+03,384.175383,0.142592
std,8.973908,9.161054e+04,1652.475629,3.890990
min,0.000000,0.000000e+00,0.000000,0.000000
25%,0.000000,9.000000e+00,48.000000,0.000000
50%,0.000000,4.300000e+01,139.000000,0.000000
75%,0.000000,1.950000e+02,376.000000,0.000000
max,916.000000,6.823332e+06,155340.000000,487.000000


In [42]:
df.dtypes

date               object
favorite_count      int64
followers_count     int64
friends_count       int64
full_text          object
retweet_count       int64
retweeted          object
screen_name        object
tweet_id           object
user_id            object
dtype: object

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16712 entries, 0 to 16711
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   date             16712 non-null  object
 1   favorite_count   16712 non-null  int64 
 2   followers_count  16712 non-null  int64 
 3   friends_count    16712 non-null  int64 
 4   full_text        16712 non-null  object
 5   retweet_count    16712 non-null  int64 
 6   retweeted        14384 non-null  object
 7   screen_name      16712 non-null  object
 8   tweet_id         16712 non-null  object
 9   user_id          16712 non-null  object
dtypes: int64(4), object(6)
memory usage: 1.3+ MB


In [44]:
#checking for the duplicated data
df.duplicated().sum()


0

In [45]:
#checking for missing values
df.isnull().sum()

date                  0
favorite_count        0
followers_count       0
friends_count         0
full_text             0
retweet_count         0
retweeted          2328
screen_name           0
tweet_id              0
user_id               0
dtype: int64

In [46]:
df["retweeted"].value_counts()

False    14384
Name: retweeted, dtype: int64

**The feature retweeted contain only false and missing values ,so the feature can be dropped**

In [47]:
df=df.drop("retweeted",axis=1)

In [48]:
df.head(10)

,date,favorite_count,followers_count,friends_count,full_text,retweet_count,screen_name,tweet_id,user_id
0,7/18/2019 22:47,0,82,219,@Mahi_2510 @swiggy_in The ultimate answer will...,0,syamantak1,1,"6,19,59,419"
1,7/18/2019 22:43,0,102,129,@SwiggyCares I hope so.,0,Bharatbbhushn,2,"14,32,84,383"
2,7/18/2019 22:37,0,102,129,@SwiggyCares I think you have the order detail...,0,Bharatbbhushn,3,"14,32,84,383"
3,7/18/2019 22:35,0,13,16,"@swiggy_in People are complaining here also, i...",0,taifkhalid01,4,"8,30,34,00,00,00,00,00,000"
4,7/18/2019 22:25,0,102,129,@SwiggyCares Do you even know the meaning of y...,0,Bharatbbhushn,5,"14,32,84,383"
5,7/18/2019 22:24,1,21,203,"Nothing new, they had the most shittiest assoc...",0,Mahi_2510,6,"9,87,01,80,00,00,00,00,000"
6,7/18/2019 22:22,0,0,2,@swiggy_in \n @SwiggyCares \n I am the owner o...,0,PaaniDaana,7,"11,51,97,00,00,00,00,00,000"
7,7/18/2019 22:22,0,3,189,@SwiggyCares Inbox check kare huzoor,0,shadowfytertoo,8,"2,87,45,57,034"
8,7/18/2019 22:19,0,102,129,@SwiggyCares Pls go through the details and so...,0,Bharatbbhushn,9,"14,32,84,383"
9,7/18/2019 22:17,0,102,129,I hope you hire educated professionals @swiggy...,1,Bharatbbhushn,10,"14,32,84,383"


In [49]:
df["favorite_count"].nunique()

52

In [50]:
df["followers_count"].nunique()

1524

In [55]:
df["friends_count"].nunique()

1505

In [54]:
df["retweet_count"].nunique()

26

In [53]:
df["screen_name"].nunique()

8617

In [52]:
df["tweet_id"].nunique()

16712

In [68]:
tweet_df = df["full_text"]
print(tweet_df.shape)
tweet_df.head(10)

(16712,)


0     2510 ultimate answer will sorry They assure that
1                                                 hope
2    think have order details number present there ...
3    People complaining here also know little disap...
4    even know meaning your tagline work want expla...
5    Nothing they most shittiest associates expect ...
6    owner restaurant named DAANA PAANI 162952 Area...
7                              Inbox check kare huzoor
8    through details solve this asap Delivery boys ...
9    hope hire educated professionals read chat thr...
Name: full_text, dtype: object

In [69]:
#timestamp splitting
df['Dates'] = pd.to_datetime(df['date']).dt.date #this first create a new column and save a new seprate data of the date
df['Time'] = pd.to_datetime(df['date']).dt.time #this create a new column and save a new seprate data of the time
df[["year", "month","day"]] = df["Dates"].astype(str).str.split("-", expand = True) #this splits the date into years months days

In [70]:

# Removing mentions
import re
def cleanTxt(text):
  text = re.sub(r'@[A-Za-z0-9]+','',text)
  #text = re.sub(r'#','',text)
  text = re.sub(r'https?:\/\/\S+','',text)
  return text

df['full_text']=df['full_text'].apply(cleanTxt)

In [71]:
#Removing punctuation other than text and number
df['full_text'].replace("[^a-zA-Z0-9#]"," ",regex=True,inplace=True)

In [72]:
#Removing the words less than 3 letter - Optional
df['full_text'] = df['full_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [73]:
tweet_df.head(10)

0     2510 ultimate answer will sorry They assure that
1                                                 hope
2    think have order details number present there ...
3    People complaining here also know little disap...
4    even know meaning your tagline work want expla...
5    Nothing they most shittiest associates expect ...
6    owner restaurant named DAANA PAANI 162952 Area...
7                              Inbox check kare huzoor
8    through details solve this asap Delivery boys ...
9    hope hire educated professionals read chat thr...
Name: full_text, dtype: object

In [74]:
df.head()

,date,favorite_count,followers_count,friends_count,full_text,retweet_count,screen_name,tweet_id,user_id,Dates,Time,year,month,day
0,7/18/2019 22:47,0,82,219,2510 ultimate answer will sorry They assure that,0,syamantak1,1,"6,19,59,419",2019-07-18,22:47:00,2019,07,18
1,7/18/2019 22:43,0,102,129,hope,0,Bharatbbhushn,2,"14,32,84,383",2019-07-18,22:43:00,2019,07,18
2,7/18/2019 22:37,0,102,129,think have order details number present there ...,0,Bharatbbhushn,3,"14,32,84,383",2019-07-18,22:37:00,2019,07,18
3,7/18/2019 22:35,0,13,16,People complaining here also know little disap...,0,taifkhalid01,4,"8,30,34,00,00,00,00,00,000",2019-07-18,22:35:00,2019,07,18
4,7/18/2019 22:25,0,102,129,even know meaning your tagline work want expla...,0,Bharatbbhushn,5,"14,32,84,383",2019-07-18,22:25:00,2019,07,18


Tokenization and stopword removal

In [78]:
import re

def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens
tweet_df_token = tweet_df.apply(lambda x: tokenize(x.lower()))

# df['reviews.text_tokenized'] = df['reviews.text'].apply(lambda x: tokenize(x.lower()))

In [79]:
tweet_df_token

0        [2510, ultimate, answer, will, sorry, they, as...
1                                                   [hope]
2        [think, have, order, details, number, present,...
3        [people, complaining, here, also, know, little...
4        [even, know, meaning, your, tagline, work, wan...
                               ...                        
16707    [they, said, delivery, ready, deliver, they, c...
16708               [deliver, karta, nahi, install, karen]
16709    [thanks, spilling, drink, waited, this, while,...
16710    [hello, there, swigggy, regularly, order, food...
16711                    [humor, help, friend, over, here]
Name: full_text, Length: 16712, dtype: object

In [80]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [81]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [82]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):

    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

tweet_df_stop =tweet_df_token.apply(lambda text: remove_stopwords(text))
tweet_df_stop

0        ['2510', 'ultimate', 'answer', 'will', 'sorry'...
1                                                 ['hope']
2        ['think', 'have', 'order', 'details', 'number'...
3        ['people', 'complaining', 'here', 'also', 'kno...
4        ['even', 'know', 'meaning', 'your', 'tagline',...
                               ...                        
16707    ['they', 'said', 'delivery', 'ready', 'deliver...
16708     ['deliver', 'karta', 'nahi', 'install', 'karen']
16709    ['thanks', 'spilling', 'drink', 'waited', 'thi...
16710    ['hello', 'there', 'swigggy', 'regularly', 'or...
16711          ['humor', 'help', 'friend', 'over', 'here']
Name: full_text, Length: 16712, dtype: object

In [85]:
from collections import Counter
cnt = Counter()
for text in tweet_df_stop.values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common

<bound method Counter.most_common of Counter({"'order',": 4736, "'your',": 3471, "'swiggy',": 3197, "'this',": 2947, "'from',": 2773, "'food',": 2742, "'delivery',": 2598, "'with',": 2418, "'that',": 2368, "'have',": 2211, "'customer',": 2163, "'they',": 2108, "'service',": 1723, "'will',": 1456, "'time',": 1334, "'what',": 1275, "'delivered',": 1233, "'refund',": 1096, "'restaurant',": 1053, "'care',": 1001, "'guys',": 994, "'just',": 954, "'after',": 933, "'ordered',": 922, "'when',": 874, "'been',": 832, "'even',": 822, "'money',": 811, "'please',": 808, "'call',": 758, "'executive',": 723, "'then',": 713, "'more',": 667, "'issue',": 655, "'support',": 625, "'take',": 621, "'received',": 621, "'deliver',": 621, "'which',": 620, "'than',": 617, "'like',": 615, "'about',": 607, "'still',": 598, "'same',": 590, "'there',": 583, "['this',": 581, "'chat',": 579, "['order',": 561, "'their',": 553, "['have',": 542, "'only',": 532, "'people',": 525, "'also',": 518, "'want',": 515, "'very',"

In [87]:
FREQWORDS = set([w for (w, wc) in cnt.most_common()])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

tweet_df_stop_freq = tweet_df_stop.apply(lambda text: remove_freqwords(text))
tweet_df_stop_freq

0         
1         
2         
3         
4         
        ..
16707     
16708     
16709     
16710     
16711     
Name: full_text, Length: 16712, dtype: object